In [ ]:
# Caso seja necessária a instalação de bibliotecas
# #pip install sty bibtexparser pandas

In [ ]:
# Bibliotecas necessárias
import bibtexparser
import pandas as pd
import numpy as np
import os

In [ ]:
# Estrutura de controle
chaves=list()

In [ ]:
# Funções
def parser_bibtex(file_bib):
    with open(file_bib,encoding='utf-8') as bibtex_file:
        try:
            bib_database = bibtexparser.load(bibtex_file)
            return pd.DataFrame(bib_database.entries)
        except:
            print('Falha no arquivo', file_bib)
            return pd.DataFrame(None)
def drop_invalid(df_tmp):
    for x in range(len(df_tmp)-1,-1,-1):
        if((df_tmp.loc[x].loc['ID'])[:8] == 'NoAuthor'):
            df_tmp.drop([int(x)], inplace=True)
        elif(df_tmp.loc[x].loc['author'] == ''):
            df_tmp.drop([int(x)], inplace=True)
    df_tmp.reset_index(inplace=True,drop=True)
    return df_tmp,len(df_tmp)
    
def drop_not_doi(df_tmp):
    for x in range(len(df_tmp)-1,-1,-1):
        if((df_tmp.loc[x].loc['doi']) == ''):
            df_tmp.drop([int(x)], inplace=True)
    df_tmp.reset_index(inplace=True,drop=True)
    return df_tmp,len(df_tmp)

def drop_year(df_tmp,ano):
    for x in range(len(df_tmp)-1,-1,-1):
        if(int(df_tmp.loc[x].loc['year']) < ano):
            df_tmp.drop([int(x)], inplace=True)
    df_tmp.reset_index(inplace=True,drop=True)
    return df_tmp,len(df_tmp)

def drop_pag(df_tmp,pag):
    for x in range(len(df_tmp)-1,-1,-1):
        #print(df_tmp.loc[x].loc['pages'])
        z = (str(df_tmp.loc[x].loc['pages'])).split('-')
        if(len(z) == 4):
            w=int(z[3])-int(z[1])
        elif(len(z) == 2):
            if((z[1].isnumeric())and(z[0].isnumeric())):
                w=int(z[1])-int(z[0])
            else:
                w=0
        else:
            w=pag+1
        if(w < 4):
            df_tmp.drop([int(x)], inplace=True)
    df_tmp.reset_index(inplace=True,drop=True)
    return df_tmp,len(df_tmp)

def drop_source(df_tmp,entrada,tipo):
    for x in range(len(df_tmp)-1,-1,-1):
        try:
            if(df_tmp.loc[x].loc['ENTRYTYPE'] in entrada):
                df_tmp.drop([int(x)], inplace=True)
            elif(df_tmp.loc[x].loc['document_type'] in tipo):
                df_tmp.drop([int(x)], inplace=True)
        except:
            None
    df_tmp.reset_index(inplace=True,drop=True)
    return df_tmp,len(df_tmp)


Definidas as chaves de busca:

1. critical infrastructure AND (vulnerabilit* AND (analys* OR Assessment OR characterization OR scan* OR tool)) AND (threat AND (shar* OR intelligent))
1. network AND (Security OR ``Vulnerabilit* Tool'' OR scan*) AND shar* AND (threat OR risk OR intelligent OR information) AND risk AND (Classification OR reduction OR mitigation)
1. threat AND (shar* OR intelligent) AND test AND (intrusion OR penetration)
1. cyber AND security AND assessment AND threat AND vulnerabilit*

Extraiu-se as referências dos resultados da busca (Export) em arquivos [base]-[chave]_[página].bib

Aplicação dos Critérios de Elegibilidade, que podem ser avaliados objetivamente com os dados da referência.

Aplicação de demais Critérios e Seleção de Estudos:
-  Utiliza Inteligência de Ameaças?
-  Identifica ou avalia vulnerabilidades?
-  Utiliza orquestração de ferramentas?
-  Utiliza automatização de processos? 
-  Mitiga vulnerabilidades em redes? 
-  Atua com medidas preventivas em Segurança Cibernética?
-  Possui relacionamento com Infraestrutura Crítica?

# Gerar Dataframe dos arquivos gerados nas Bases de busca

In [ ]:
# Encontrar arquivos
y=0
for x in os.listdir():
    if('.bib' in x):
        chaves.append(['df_' + x.split(".")[0].replace('-','_')])
        globals()[chaves[y][0]] = parser_bibtex(x)
        chaves[y].append(len(globals()[chaves[y][0]]))
        y+=1
del x,y
chaves

# Aplicar Critérios de Elegibilidade

In [ ]:
# Entrada Inválida
for y,chave in enumerate(chaves):
    globals()[chave[0]],aux = drop_invalid(globals()[chave[0]])
    chaves[y].append(aux)
del aux,y
chaves

In [ ]:
# Ano
ano=2010
for y,chave in enumerate(chaves):
    globals()[chave[0]],aux = drop_year(globals()[chave[0]],ano)
    chaves[y].append(aux)
del aux,y
chaves

In [ ]:
# Número de páginas
pag = 3
for y,chave in enumerate(chaves):
    globals()[chave[0]],aux = drop_pag(globals()[chave[0]],pag)
    chaves[y].append(aux)
del aux,y
chaves

In [ ]:
# Não indexado
for y,chave in enumerate(chaves):
    globals()[chave[0]],aux = drop_not_doi(globals()[chave[0]])
    chaves[y].append(aux)
del aux,y
chaves

In [ ]:
# Fonte
entrada = ['book','inbook']
tipo = ['Book Chapter','Article in Press','Note']
for y,chave in enumerate(chaves):
    globals()[chave[0]],aux = drop_source(globals()[chave[0]],entrada,tipo)
    chaves[y].append(aux)
del aux,y
chaves

In [ ]:
# Concatenar por Base de Conhecimento e Remover Duplicatas
df_ieee = pd.DataFrame(None)
df_scopus = pd.DataFrame(None)
for y,chave in enumerate(chaves):
    if('ieee' in chave[0]):
        df_ieee = pd.concat([df_ieee,globals()[chave[0]]], ignore_index=True)
    elif(('scopus')in chave[0]):
        df_scopus = pd.concat([df_scopus,globals()[chave[0]]], ignore_index=True)
    del globals()[chave[0]]
bases = [['df_ieee',len(df_ieee)],['df_scopus',len(df_scopus)]]
for y,base in enumerate(bases):
    globals()[base[0]].drop_duplicates(subset=['ID','title'],inplace=True)
    globals()[base[0]].reset_index(inplace=True,drop=True)
    bases[y].append(len(globals()[base[0]]))
    
del y
bases

In [ ]:
# Concatenar por Base de Conhecimento e Remover Duplicatas
df = pd.concat([df_ieee,df_scopus], ignore_index=True)
pesquisa=['dados',len(df)]
df.drop_duplicates(subset=['doi','title'],inplace=True)
df.reset_index(inplace=True,drop=True)
pesquisa.append(len(df))

del df_ieee,df_scopus
pesquisa

# Relatório de aplicação

# Gerar Referências padrão ITA para implementação no LATEX

In [ ]:
dados = {
    'article': ['ENTRYTYPE','ID','year', 'pages', 'volume', 'journal', 'title','author'],
    'book': ['ENTRYTYPE','ID','year','pages', 'volume', 'title',
       'author', 'ENTRYTYPE', 'ID', 'url', 'publisher', 'issn',
       'isbn', 'address', 'booktitle'],
    'inproceedings': ['ENTRYTYPE','ID','address','author','conference-location','conference-number','conference-year',
        'organization','pages','publisher','subtitle','title','url','year'],
    'lei': ['ENTRYTYPE','ID','address','journal','month','organization','volume','title','number','pages','url','urlaccessdate','year'],
    'mastersthesis': ['ENTRYTYPE','ID','author', 'pages', 'title', 'year', 'year-presented', 'type', 'publisher', 'school', 'address'],
    'misc': ['ENTRYTYPE','ID','address','author','edition','note','publisher','subtitle','title','url','urlaccessdate','volume','year'],
    'techreport': ['ENTRYTYPE','ID','address','author','booktitle','edition','isbn','pages','publisher','subtitle','title','url','volume','year']
}

In [ ]:
from bibtexparser.bwriter import BibTexWriter
from bibtexparser.bibdatabase import BibDatabase

writer = BibTexWriter()
writer.indent = ' '     # indent entries with 4 spaces instead of one
writer.comma_first = False  # place the comma at the beginning of the line

for entry in (dados.keys()):
    globals()['df_' + entry] = df.loc[(entry == df['ENTRYTYPE'])].drop(columns=['abstract','keywords'],axis=1)
    globals()['df_' + entry].reset_index(inplace=True,drop=True)
    globals()['df_' + entry] = globals()['df_' + entry].replace(np.nan, '', regex=True)
    globals()[entry] = globals()['df_' + entry].to_dict(orient="records")
    globals()['db_' + entry] = BibDatabase()
    globals()['db_' + entry].entries = globals()[entry]

    with open('referencias.bib', 'a',encoding='utf-8') as bibfile:
        bibfile.write(writer.write(globals()['db_' + entry]))
    
    del globals()['df_' + entry], globals()['db_' + entry],globals()[entry]
del writer, bibfile